In [54]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [13]:
import pandas as pd

In [14]:
top_model_weights_path = '/home/yk4we/SYS6016/seeing-through/bottleneck_fc_model.h5'

from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
# define train and validation generators here

In [16]:
data = pd.read_csv('/home/yk4we/SYS6016/fer2013_and_plus.csv')

In [17]:
import pandas as pd
import cv2
import numpy as np

In [18]:
dataset_path = '/home/yk4we/SYS6016/fer2013_and_plus.csv'
image_size=(48,48)
def load_fer2013():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).to_numpy()
    return faces, emotions
 
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x
 
faces, emotions = load_fer2013()
faces = preprocess_input(faces)

In [19]:
def split_data(x, y, validation_split=.2):
    num_samples = len(x)
    num_train_samples = int((1 - validation_split)*num_samples)
    train_x = x[:num_train_samples]
    train_y = y[:num_train_samples]
    val_x = x[num_train_samples:]
    val_y = y[num_train_samples:]
    train_data = (train_x, train_y)
    val_data = (val_x, val_y)
    return train_data, val_data


In [20]:
trained_data, val_data = split_data(faces, emotions)

In [21]:
x_train = trained_data[0]
y_train = trained_data[1]

x_val = val_data[0]
y_val = val_data[1]

In [32]:
x_train.shape

(28709, 48, 48, 1)

In [50]:
x_train = x_train.reshape(x_train.shape[0], 48, 48,1)
x_train = x_train.astype('float32')
x_val = x_val.reshape(x_val.shape[0], 48, 48,1)
x_val = x_val.astype('float32')

In [51]:
#initilize parameters
num_classes = 7 
width, height = 48, 48
num_epochs = 50
batch_size = 64
num_features = 64

In [52]:
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)


es = EarlyStopping(monitor='val_loss', patience = 10, mode = 'min', restore_best_weights=True)

history = model.fit_generator(data_generator.flow(x_train, y_train, batch_size),
                                steps_per_epoch=len(x_train) / batch_size,
                                epochs=num_epochs,
                                verbose=2, 
                                callbacks = [es],
                                validation_data=(x_val, y_val))

  ...
    to  
  ['...']


ValueError: Error when checking input: expected resnet50_input to have shape (48, 48, 3) but got array with shape (48, 48, 1)

In [37]:
img_height, img_width = 48, 48

# Parameters
num_classes         = 7     # ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']
epochs_top_layers   = 5
epochs_all_layers   = 100
batch_size          = 128

In [38]:
train_datagen = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)

# Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
    # x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
    #               it should have value 3
    # y:            Labels
    # batch_size:   Int (default: 32)
train_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size  = batch_size)

In [39]:
val_datagen = ImageDataGenerator()
validation_generator = val_datagen.flow(
    x_val,
    y_val,
    batch_size  = batch_size)

## MODEL: ResNet50 Finetuned

In [54]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(48,48,3))

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights(top_model_weights_path)

In [47]:
model = Sequential()
model.add(base_model)
model.add(top_model)

for layer in model.layers[:50]:
    layer.trainable = False

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [28]:
dataset_name = 'fer2013_and_plus'

In [42]:
log_file_path = 'logs/' + dataset_name + '_training.log'
csv_logger = CSVLogger(log_file_path, append=False)

#early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(100/4), verbose=1)

trained_models_path = 'model_weights_resnet/' + dataset_name + '_resnet50_finetune'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, monitor='val_accuracy', verbose=1, save_best_only=True)

callbacks = [model_checkpoint, csv_logger, reduce_lr]

In [43]:
resnet_finetune = model.fit_generator(train_generator, steps_per_epoch=171098 // batch_size, 
                                 epochs=100, validation_data=validation_generator, 
                                 validation_steps=53742 // batch_size, verbose=1,
                                 callbacks=callbacks)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1336 steps, validate for 419 steps
Epoch 1/100
   1/1336 [..............................] - ETA: 18:59

KeyError: 'val_accuracy'